In [16]:
import pingouin as pg
import pandas as pd

In [19]:
data = pd.read_csv("task-2-data.csv", sep=",")
data

,factor1,factor2,minutes
0,rano,ticho,6
1,rano,ticho,8
2,rano,ticho,11
3,rano,hudba,7
4,rano,hudba,8
5,rano,hudba,12
6,rano,hudba,10
7,rano,hluk,8
8,rano,hluk,7
9,rano,hluk,20


In [20]:
data.anova(dv="minutes", between=["factor1", "factor2"]).round(3)

,Source,SS,DF,MS,F,p-unc,np2
0,factor1,16.562,2.0,8.281,0.566,0.574,0.039
1,factor2,290.452,3.0,96.817,6.621,0.002,0.415
2,factor1 * factor2,47.305,6.0,7.884,0.539,0.774,0.104
3,Residual,409.417,28.0,14.622,NaN,NaN,NaN
